In [1]:
%load_ext autoreload
% autoreload 2

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from tensorboardX import SummaryWriter
from sklearn.metrics import f1_score
from tqdm import tqdm_notebook

from dataset import MNISTDataset
from models import CNN

In [3]:
def imshow(img, title=None):
    plt.imshow(img/(np.max(img)-np.min(img)))
    if title is not None:
        plt.title(title)
    plt.show()

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [5]:
data_dir = 'data/'
validation_split = 0.2
batch_size = 64
num_workers = 3
image_size = (64, 64)
num_classes = 10
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])

In [6]:
train_set = MNISTDataset(os.path.join(data_dir, 'train_small.npz'), transform=transform)
val_set = MNISTDataset(os.path.join(data_dir, 'val_small.npz'), transform=transform)

In [7]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=num_workers)

In [8]:
dropout = 0.275

model = CNN(input_shape=(1, *image_size), dropout=dropout, num_classes=num_classes)
model = model.to(device)

criterion = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(model.parameters(), weight_decay=0.005)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, verbose=True)

num_epochs = 50
train_steps_per_epoch = int((len(train_set)+1)/batch_size)
val_steps_per_epoch = int((len(val_set)+1)/batch_size)
models_dir = 'models/'
model_name = 'CNN-small-adam'

In [9]:
writer = SummaryWriter(comment=model_name)

best_val_loss = float('inf')
print('Started Training')
for epoch in range(num_epochs):
    
    model.train(True)
    train_loss = 0.0
    for i, data in tqdm_notebook(enumerate(train_loader, 0),
                                 total=train_steps_per_epoch,
                                 desc='Epoch {}: '.format(epoch)):
        x,y = data.values()
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        p = model(x)
        loss = criterion(p, y.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if i >= train_steps_per_epoch:
            break
    writer.add_scalar('train_loss', train_loss / (len(train_set)/batch_size), epoch)
    
    model.eval()
    val_loss = 0.0
    y_true = None
    y_pred = None
    for i, data in enumerate(val_loader, 0):
        x,y = data.values()
        x = x.to(device)
        y = y.to(device)
        p = model(x)
        loss = criterion(p, y.float())
        val_loss += loss.item()
        if y_true is None:
            y_true = y
        else:
            y_true = torch.cat([y_true, y], dim=0)
            
        if y_pred is None:
            y_pred = y.byte()
        else:
            y_pred = torch.cat([y_pred, (torch.sigmoid(p).data > 0.5)], dim=0)

        if i >= val_steps_per_epoch:
            break
            
    y_true = torch.transpose(y_true, 0, 1).cpu().numpy()
    y_pred = torch.transpose(y_pred, 0, 1).cpu().numpy()

    f1score = []

    for i in range(num_classes):
        f1score.append(f1_score(y_true[i], y_pred[i]))

    avg_f1score = np.average(f1score)
    
    writer.add_scalar('validation_loss', val_loss / (len(val_set)/batch_size), epoch)
    writer.add_scalar('average f1-score on val. set', avg_f1score, epoch)
    
    print('train_loss: %.5f, validation_loss: %.5f, validation_f1-score: %.3f' % 
          (train_loss / (len(train_set)/batch_size), val_loss / (len(val_set)/batch_size), avg_f1score))
    
    if val_loss < best_val_loss:
        torch.save(model.state_dict(), os.path.join(models_dir, model_name))
        best_val_loss = val_loss
        print('Model Saved!')
        
    train_loss = 0.0
    val_loss = 0.0
            
print('Finished Training')

Started Training



train_loss: 0.37507, validation_loss: 0.33686, validation_f1-score: 0.062
Model Saved!



train_loss: 0.34106, validation_loss: 0.32998, validation_f1-score: 0.062
Model Saved!



train_loss: 0.32978, validation_loss: 0.31237, validation_f1-score: 0.062
Model Saved!



train_loss: 0.31459, validation_loss: 0.29872, validation_f1-score: 0.062
Model Saved!


...


train_loss: 0.21737, validation_loss: 0.20552, validation_f1-score: 0.405



train_loss: 0.21405, validation_loss: 0.20270, validation_f1-score: 0.392
Model Saved!



train_loss: 0.21559, validation_loss: 0.20698, validation_f1-score: 0.385



train_loss: 0.21512, validation_loss: 0.20277, validation_f1-score: 0.392



train_loss: 0.21313, validation_loss: 0.19659, validation_f1-score: 0.425
Model Saved!
Finished Training


In [ ]:
torch.save(model.state_dict(), os.path.join(models_dir, 'cnn_best'))

In [ ]:
model.load_state_dict(torch.load(os.path.join(models_dir,model_name)))

In [ ]:
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for data in val_loader:
        images, labels = data.values()
        images = images.cuda(device)
        outputs = model(images)
        y_true += list(labels.numpy())
        y_pred += list((torch.sigmoid(outputs).data > 0.5 ).cpu().numpy())
        
y_true = np.asarray(y_true).T
y_pred = np.asarray(y_pred).T

f1score = []

for i in range(num_classes):
    f1score.append(f1_score(y_true[i], y_pred[i]))
    
avg_f1score = np.average(f1score)
    
print(f1score)
print('Average f1-score = {}'.format(avg_f1score))



In [ ]:
imshow(val_set[0]['x'][0].numpy())